In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import json

In [3]:
with open('huggingface-3mixed-train-test.json') as fopen:
    data = json.load(fopen)

In [4]:
data = [d for d in data['train'] if 'malay/' in d]
len(data)

609

In [5]:
from multiprocessing import Pool
from glob import glob
import os
import requests
import shutil

In [7]:
def download_file_cloud(url, filename):
    try:
        while True:
            r = requests.get(url, stream=True)
            total_size = int(r.headers['content-length'])
            version = int(r.headers.get('X-Bz-Upload-Timestamp', 0))
            try:
                local_size = os.path.getsize(filename)
                if local_size == total_size:
                    print(f'{filename} local size matched with cloud size')
                    return version
            except Exception as e:
                print(e)
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename, 'wb') as f:
                for data in r.iter_content(chunk_size=1_048_576):
                    f.write(data)
            break
    except Exception as e:
        print(f'download_file_cloud error: {e}')

In [8]:
def get_dataset(files, directory='tfrecord'):
    os.makedirs(directory, exist_ok=True)
    shutil.rmtree(directory)
    files_to_download = []
    for f in files:
        filename = os.path.join(directory, '-'.join(f.split('/')[-2:]))
        files_to_download.append((f, filename))

    pool = Pool(processes=len(files))
    pool.starmap(download_file_cloud, files_to_download)
    pool.close()
    pool.join()
    tfrecords = glob(f'{directory}/*.tfrecord')
    return tfrecords

In [9]:
def parse(serialized_example):

    data_fields = {
        'waveforms': tf.compat.v1.VarLenFeature(tf.float32),
        'targets': tf.compat.v1.VarLenFeature(tf.int64),
        'targets_length': tf.compat.v1.VarLenFeature(tf.int64),
        'lang': tf.compat.v1.VarLenFeature(tf.int64),
    }
    features = tf.compat.v1.parse_single_example(
        serialized_example, features=data_fields
    )
    for k in features.keys():
        features[k] = features[k].values

    keys = list(features.keys())
    for k in keys:
        if k not in ['waveforms', 'waveforms_length', 'targets']:
            features.pop(k, None)

    return features

In [10]:
files = data
batch_files = 10
directory = 'tfrecord'
num_cpu_threads = 4
thread_count = 12
sr = 16000

In [24]:
from tqdm import tqdm

lengths = []
for i in tqdm(range(0, len(files), batch_files)):
    b = files[i: i + batch_files]
    tfrecords = get_dataset(b, directory=directory)
    d = tf.data.Dataset.from_tensor_slices(tf.constant(tfrecords))
    d = d.shuffle(buffer_size=len(tfrecords))
    cycle_length = min(num_cpu_threads, len(tfrecords))
    d = d.interleave(
        tf.data.TFRecordDataset,
        cycle_length=cycle_length,
        block_length=thread_count)
    d = d.map(parse, num_parallel_calls=num_cpu_threads)
    d = d.as_numpy_iterator()
    while True:
        try:
            r = next(d)
            lengths.append(len(r['waveforms']) / sr)
        except Exception as e:
            print(e)
            break

  0%|          | 0/61 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'tfrecord/malay-5-2.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-4-68.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-0-2.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-4-38.tfrecord'[Errno 2] No such file or directory: 'tfrecord/malay-2-33.tfrecord'

[Errno 2] No such file or directory: 'tfrecord/malay-3-36.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-2-25.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-2-62.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-6-25.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-6-48.tfrecord'


  2%|▏         | 1/61 [02:50<2:50:28, 170.47s/it]


[Errno 2] No such file or directory: 'tfrecord/malay-5-11.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-5-13.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-2-42.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-0-10.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-6-28.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-7-29.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-4-20.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-5-58.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-7-52.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-5-50.tfrecord'


  3%|▎         | 2/61 [05:46<2:50:47, 173.69s/it]

...

 97%|█████████▋| 59/61 [4:27:01<09:26, 283.00s/it]


[Errno 2] No such file or directory: 'tfrecord/malay-5-12.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-2-38.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-6-50.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-4-24.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-6-71.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-0-14.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-0-17.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-4-31.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-4-73.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-3-40.tfrecord'


 98%|█████████▊| 60/61 [4:31:39<04:41, 281.52s/it]


[Errno 2] No such file or directory: 'tfrecord/malay-6-26.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-7-23.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-0-62.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-6-63.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-3-37.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-1-47.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-2-58.tfrecord'
[Errno 2] No such file or directory: 'tfrecord/malay-3-1.tfrecord'[Errno 2] No such file or directory: 'tfrecord/malay-7-9.tfrecord'



100%|██████████| 61/61 [4:35:52<00:00, 271.35s/it]

In [12]:
import numpy as np

In [25]:
np.mean(lengths)

5.8575222180546636

In [26]:
np.min(lengths)

0.3453125

In [27]:
np.max(lengths)

18.0

In [28]:
np.percentile(lengths, [10, 25, 50, 75, 90])

array([4.056  , 5.07   , 5.36825, 6.12275, 8.04   ])

In [29]:
np.sum(lengths) / 60 / 60

2398.3592180034716